In [42]:
import os, sys
import glob
import pandas as pd
import numpy as np
import h5py
from scipy import io as sio
import matplotlib.pyplot as plt
from core.utils import gat, chen_estimate
import torch

In [16]:
!ls 

Scatter_plot_for_pge-net_estimation_no_sampling-Copy1.ipynb
Scatter_plot_for_pge-net_estimation_no_sampling.ipynb
Scatter_plot_for_pge-net_MAE.ipynb


In [22]:
!ls ../../data/Fivek_dataset/test_fivek_rawRGB_alpha_0.01_beta_0.0002.hdf5

../../data/Fivek_dataset/test_fivek_rawRGB_alpha_0.01_beta_0.0002.hdf5


In [5]:
!ls ../../result_data//230127*

../../result_data//230127_PGE_Net_RawRGB_fivek_alpha_0.01_beta_0.0002_Noise_est_cropsize_200_vst_MAE_result
../../result_data//230127_PGE_Net_RawRGB_fivek_alpha_0.01_beta_0.0002_Noise_est_cropsize_200_vst_MSE_result
../../result_data//230127_PGE_Net_RawRGB_fivek_alpha_0.01_beta_0.02_Noise_est_cropsize_200_vst_MAE_result
../../result_data//230127_PGE_Net_RawRGB_fivek_alpha_0.01_beta_0.02_Noise_est_cropsize_200_vst_MSE_result


dict_keys(['__header__', '__version__', '__globals__', 'tr_loss_arr', 'a_arr', 'b_arr'])

In [48]:
MSE_result_data['a_arr'].shape

(1, 200)

In [60]:
alpha = 0.01
beta = 0.02



In [61]:
source = noisy_images[0]
source = torch.from_numpy(source.reshape(1,source.shape[0],source.shape[1])).float()

In [62]:
source.shape, source.dtype

(torch.Size([1, 512, 512]), torch.float32)

In [63]:
noisy_img = torch.Tensor(noisy_images[0])
noisy_img.shape, noisy_img.dtype

(torch.Size([512, 512]), torch.float32)

In [97]:
def return_result(ref_item,item, func):
    return item if ref_item is None else func(ref_item,item)
def update_data(info, item):
    info['min'] = return_result(info['min'], item,min)
    info['max'] = return_result(info['max'], item,max)
    info['average'].append(item)
    return info
def return_average(item):
    item['MSE']['average'] = np.average(item['MSE']['average'])
    item['MAE']['average'] = np.average(item['MAE']['average'])
    item['real']['average'] = np.average(item['real']['average'])
    return item

In [122]:
from copy import deepcopy as dcpy
print_log = True

whole_metric = {}
which_metric = 'beta' #'gaussian' 'alpha' 'beta'
for alpha, beta in [[0.01,0.02], [0.01,0.0002]]:
    print(f"=========== alpha : {alpha} beta : {beta} ===========")
    MSE_filename = f"../../result_data/230127_PGE_Net_RawRGB_fivek_alpha_{alpha}_beta_{beta}_Noise_est_cropsize_200_vst_MSE_result"
    MAE_filename = f"../../result_data/230127_PGE_Net_RawRGB_fivek_alpha_{alpha}_beta_{beta}_Noise_est_cropsize_200_vst_MAE_result"
    MSE_result_data = sio.loadmat(MSE_filename)
    MAE_result_data = sio.loadmat(MAE_filename)
    MSE_result_data.keys()
    test_dataset = f"../../data/Fivek_dataset/test_fivek_rawRGB_alpha_{alpha}_beta_{beta}.hdf5"
    with h5py.File(test_dataset,'r') as f:
        clean_images = np.array(f['clean_images'])
        noisy_images = np.array(f['noisy_images'])
    metric = {"min" : None, "max" : None, "average" : []}
    total_metric = {"MSE": dcpy(metric), "MAE" : dcpy(metric), "real" : dcpy(metric)}
    for idx, (noisy_img, clean_img) in enumerate(zip(noisy_images,clean_images)):
        MSE_alpha, MSE_beta = MSE_result_data['a_arr'][0,idx],MSE_result_data['b_arr'][0,idx]
        MAE_alpha, MAE_beta = MAE_result_data['a_arr'][0,idx],MAE_result_data['b_arr'][0,idx]
        # print("======")
        if print_log is True:
            print(f"Real               : alpha : {alpha:.8f} sigma : {beta:.8f}")
            print(f"estimated with MSE : alpha : {MSE_alpha:.8f} sigma : {MSE_beta:.8f}")
            print(f"estimated with MAE : alpha : {MAE_alpha:.8f} sigma : {MAE_beta:.8f}")
        noisy_img = torch.from_numpy(noisy_img)

        MAE_transformed_with_pge = gat(noisy_img, MAE_alpha, MAE_beta,0)
        MAE_estimated_gaussian_noise_level = chen_estimate(MAE_transformed_with_pge).cpu().item()

        MSE_transformed_with_pge = gat(noisy_img, MSE_alpha, MSE_beta,0)
        MSE_estimated_gaussian_noise_level = chen_estimate(MSE_transformed_with_pge).cpu().item()

        real_transformed_with_pge = gat(noisy_img, alpha, beta,0)
        real_estimated_gaussian_noise_level = chen_estimate(real_transformed_with_pge).cpu().item()
        if which_metric == 'gaussian':
            update_data(total_metric['MSE'],MSE_estimated_gaussian_noise_level)
            update_data(total_metric['MAE'],MAE_estimated_gaussian_noise_level)
            update_data(total_metric['real'],real_estimated_gaussian_noise_level)
        elif which_metric == 'alpha':
            update_data(total_metric['MSE'],MSE_alpha)
            update_data(total_metric['MAE'],MAE_alpha)
            update_data(total_metric['real'],alpha)
        elif which_metric == 'beta':
            update_data(total_metric['MSE'],MSE_beta)
            update_data(total_metric['MAE'],MAE_beta)
            update_data(total_metric['real'],beta)
        else :
            raise ValueError("No options for ",which_metric)
        if print_log is True:
            print(f"{idx}th image Gaussian variance estimation result")
            print(idx, f"MSE        : {MSE_estimated_gaussian_noise_level:.8f}")
            print(idx, f"MAE        : {MAE_estimated_gaussian_noise_level:.8f}")
            print(idx, f"Real param : {real_estimated_gaussian_noise_level:.8f}")
            print("======")
            break
    total_metric = return_average(total_metric)
    whole_metric[f"{alpha}/{beta}"] = total_metric

=========== alpha : 0.01 beta : 0.02 ===========
Real               : alpha : 0.01000000 sigma : 0.02000000
estimated with MSE : alpha : 0.01559884 sigma : 0.00421521
estimated with MAE : alpha : 0.01574573 sigma : 0.00390665
0th image Gaussian variance estimation result
0 MSE        : 1.28810477
0 MAE        : 1.31024635
0 Real param : 0.81218469
=========== alpha : 0.01 beta : 0.0002 ===========
Real               : alpha : 0.01000000 sigma : 0.00020000
estimated with MSE : alpha : 0.01010000 sigma : 0.00005100
estimated with MAE : alpha : 0.00996626 sigma : 0.00002288
0th image Gaussian variance estimation result
0 MSE        : 2.52208948
0 MAE        : 2.62160635
0 Real param : 2.29323888


In [120]:
alpha = 0.01
beta = 0.02
print(f"==== dataset ====")
print("Fivek")
print(f"==== parameter =====")
print(f"alpha      : {alpha:.4f}")
print(f"sigma      : {beta:.4f}")
if which_metric == 'gaussian' :
    print("====estimated gaussian variance (supposed to be 1)====")
elif which_metric == 'alpha':
    print("==== estimated alpha====")
elif which_metric == 'beta':
    print("==== estimated beta====")
print(f"MSE        : {whole_metric['0.01/0.02']['MSE']['average']:.8f} ({whole_metric['0.01/0.02']['MSE']['min']:.8f}~{whole_metric['0.01/0.02']['MSE']['max']:.8f})")
print(f"MAE        : {whole_metric['0.01/0.02']['MAE']['average']:.8f} ({whole_metric['0.01/0.02']['MAE']['min']:.8f}~{whole_metric['0.01/0.02']['MAE']['max']:.8f})")
print( f"Real param : {whole_metric['0.01/0.02']['real']['average']:.8f} ({whole_metric['0.01/0.02']['real']['min']:.8f}~{whole_metric['0.01/0.02']['real']['max']:.8f})")
# whole_metric['0.01/0.02']['MSE']

==== dataset ====
Fivek
==== parameter =====
alpha      : 0.0100
sigma      : 0.0200
==== estimated beta====
MSE        : 0.00437907 (0.00406262~0.00531220)
MAE        : 0.00384582 (0.00375222~0.00563847)
Real param : 0.02000000 (0.02000000~0.02000000)
